In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2 as cv
import os
from pathlib import Path

In [4]:
img_labels = {'cats':0,
             'dogs':1,
             'horses':2,
             'humans':3
             }

In [5]:
ps = Path('../SVM/images/')
base_dir = ps.glob('*')

In [6]:
###Reading image files from the folders
img_data = []
img_label = []
for folder in base_dir:
    ps = Path(folder)
    folder_name = str(folder).split('\\')[-1]
    ps = Path(str(folder))
    files = ps.glob('*')
    for file in files:
        print(file)
        img = cv.imread(str(file))
        img = cv.resize(img,(50,50))
        img_data.append(img)
        img_label.append(img_labels[folder_name])


..\SVM\images\cats\cat.1.jpg
..\SVM\images\cats\cat.10.jpg
..\SVM\images\cats\cat.100.jpg
..\SVM\images\cats\cat.101.jpg
..\SVM\images\cats\cat.102.jpg
..\SVM\images\cats\cat.103.jpg
..\SVM\images\cats\cat.104.jpg
..\SVM\images\cats\cat.105.jpg
..\SVM\images\cats\cat.106.jpg
..\SVM\images\cats\cat.107.jpg
..\SVM\images\cats\cat.108.jpg
..\SVM\images\cats\cat.109.jpg
..\SVM\images\cats\cat.11.jpg
..\SVM\images\cats\cat.110.jpg
..\SVM\images\cats\cat.111.jpg
..\SVM\images\cats\cat.112.jpg
..\SVM\images\cats\cat.113.jpg
..\SVM\images\cats\cat.114.jpg
..\SVM\images\cats\cat.115.jpg
..\SVM\images\cats\cat.116.jpg
..\SVM\images\cats\cat.117.jpg
..\SVM\images\cats\cat.118.jpg
..\SVM\images\cats\cat.119.jpg
..\SVM\images\cats\cat.12.jpg
..\SVM\images\cats\cat.120.jpg
..\SVM\images\cats\cat.121.jpg
..\SVM\images\cats\cat.122.jpg
..\SVM\images\cats\cat.123.jpg
..\SVM\images\cats\cat.124.jpg
..\SVM\images\cats\cat.125.jpg
..\SVM\images\cats\cat.126.jpg
..\SVM\images\cats\cat.127.jpg
..\SVM\images

..\SVM\images\dogs\dog.181.jpg
..\SVM\images\dogs\dog.182.jpg
..\SVM\images\dogs\dog.183.jpg
..\SVM\images\dogs\dog.184.jpg
..\SVM\images\dogs\dog.185.jpg
..\SVM\images\dogs\dog.186.jpg
..\SVM\images\dogs\dog.187.jpg
..\SVM\images\dogs\dog.188.jpg
..\SVM\images\dogs\dog.189.jpg
..\SVM\images\dogs\dog.19.jpg
..\SVM\images\dogs\dog.190.jpg
..\SVM\images\dogs\dog.191.jpg
..\SVM\images\dogs\dog.192.jpg
..\SVM\images\dogs\dog.193.jpg
..\SVM\images\dogs\dog.194.jpg
..\SVM\images\dogs\dog.195.jpg
..\SVM\images\dogs\dog.196.jpg
..\SVM\images\dogs\dog.197.jpg
..\SVM\images\dogs\dog.198.jpg
..\SVM\images\dogs\dog.199.jpg
..\SVM\images\dogs\dog.2.jpg
..\SVM\images\dogs\dog.20.jpg
..\SVM\images\dogs\dog.200.jpg
..\SVM\images\dogs\dog.201.jpg
..\SVM\images\dogs\dog.202.jpg
..\SVM\images\dogs\dog.21.jpg
..\SVM\images\dogs\dog.22.jpg
..\SVM\images\dogs\dog.23.jpg
..\SVM\images\dogs\dog.24.jpg
..\SVM\images\dogs\dog.25.jpg
..\SVM\images\dogs\dog.26.jpg
..\SVM\images\dogs\dog.27.jpg
..\SVM\images\dogs\

..\SVM\images\humans\rider-147.jpg
..\SVM\images\humans\rider-148.jpg
..\SVM\images\humans\rider-149.jpg
..\SVM\images\humans\rider-15.jpg
..\SVM\images\humans\rider-150.jpg
..\SVM\images\humans\rider-151.jpg
..\SVM\images\humans\rider-152.jpg
..\SVM\images\humans\rider-153.jpg
..\SVM\images\humans\rider-154.jpg
..\SVM\images\humans\rider-155.jpg
..\SVM\images\humans\rider-156.jpg
..\SVM\images\humans\rider-157.jpg
..\SVM\images\humans\rider-158.jpg
..\SVM\images\humans\rider-159.jpg
..\SVM\images\humans\rider-16.jpg
..\SVM\images\humans\rider-160.jpg
..\SVM\images\humans\rider-161.jpg
..\SVM\images\humans\rider-162.jpg
..\SVM\images\humans\rider-163.jpg
..\SVM\images\humans\rider-164.jpg
..\SVM\images\humans\rider-165.jpg
..\SVM\images\humans\rider-166.jpg
..\SVM\images\humans\rider-167.jpg
..\SVM\images\humans\rider-168.jpg
..\SVM\images\humans\rider-169.jpg
..\SVM\images\humans\rider-17.jpg
..\SVM\images\humans\rider-170.jpg
..\SVM\images\humans\rider-171.jpg
..\SVM\images\humans\ri

In [7]:
img_label = np.asarray(img_label)
img_data=np.asarray(img_data,dtype='uint8')

In [8]:
###randomly shuffle the data

import random

combined = list(zip(img_data,img_label))
random.shuffle(combined)
img_data[:],img_label[:] = zip(*combined)

In [9]:
###converting image to 1-d so that we can apply algos
m = img_data.shape[0]
img_data = img_data.reshape((m,-1))
img_data.shape

(808, 7500)

## ClassWise divison of data

In [10]:
num_class = len(np.unique(img_label))
def class_divison(img_data,img_label):
    data = {}
    for i in range(num_class):
        data[i]=[]
    for i in range(img_data.shape[0]):
        data[img_label[i]].append(img_data[i])
    for keys in data:
        data[keys] = np.asarray(data[keys])
    
    return data
        

In [11]:
data = class_divison(img_data,img_label)

In [12]:
print(data[0].shape,data[1].shape,data[2].shape,data[3].shape)

(202, 7500) (202, 7500) (202, 7500) (202, 7500)


In [13]:
###Combining data of two classes in a single matrix
def DataPair(d1,d2):
    num_samples = d1.shape[0]+d1.shape[0]
    num_feature = d1.shape[1]
    data_pair = np.zeros((num_samples,num_feature))
    data_label = np.zeros((num_samples,))
    
    data_pair[:d1.shape[0],:]=d1[:,:]
    data_pair[d1.shape[0]:,:]=d2[:,:]
    
    data_label[:d1.shape[0]] = 1
    data_label[d1.shape[0]:] = -1
    
    return data_pair,data_label
    
    

In [14]:
###SVM algorithm
class SVM():
    def __init__ (self,c=1):
        self.c = c
        self.w = 0
        self.b = 0
    
    
    def hingeLoss(self,W,B,C,X,Y):
        loss = 0.0
        loss+=0.5*np.dot(W,W.T)
        for i in range(X.shape[0]):
            ti = 1-Y[i]*(np.dot(W,X[i].T)+B)
            loss+=C*max(0,ti)
        return loss[0][0]
    
    
    def fit(self,X,Y,batch_size=100,max_itr=1000,learning_val=0.0001):
        no_featues = X.shape[1]
        no_sample = X.shape[0]
        c = self.c
        n = learning_val
        
        
        W = np.zeros((1,no_featues))
        bias = 0
        
        losses= []

        for i in range(max_itr):
            
            l = self.hingeLoss(W,bias,c,X,Y)
            losses.append(l)
            
            ids = np.arange(no_sample)
            np.random.shuffle(ids)
            
            for batch_start in range(0,no_sample,batch_size):
                gradw = 0
                gradb = 0
                
                for j in range(batch_start,batch_start+batch_size):
                    if j<no_sample:
                        i = ids[j]
                    
                        ti = Y[i]*(np.dot(W,X[i].T)+bias)
                        if ti>1:
                            gradw+=0
                            gradb+=0
                        else:
                            gradw+=(c*Y[i]*X[i])
                            gradb+=c*Y[i]
            
                W = W -n*W + n*gradw
                bias = bias + n*gradb
            
        self.w = W[0]
        self.b = bias
        return self.w,bias

In [15]:
###Performing one to one multi-class classificaion
def svmTrainer(x,y):
    data = class_divison(x,y)
    svm_classifier = {}
    for i in range(num_class):
        svm_classifier[i]={}
        for j in range(i+1,num_class):
            data_pair,data_label = DataPair(data[i],data[j])
            svm = SVM()
            svm_classifier[i][j] = svm.fit(data_pair,data_label)
        
    return svm_classifier

In [16]:
svm_classifier = svmTrainer(img_data,img_label)

In [17]:

def binaryclassifier(x,w,b):
    z = (np.dot(x,w.T)+b)
    if z>=0:
        return 1
    else:
        return -1

###predicting the class to which given data belongs
def predict(x):
    count = np.zeros((num_class,))
    for i in range(num_class):
        for j in range(i+1,num_class):
            w,b = svm_classifier[i][j]
            z = binaryclassifier(x,w,b)
            if z==1:
                count[i]+=1
            else:
                count[j]+=1
    return np.argmax(count)

In [18]:
predict(img_data[684])

0

In [19]:
def accuracy(img_data,img_label):
    cnt=0
    for i in range(img_data.shape[0]):
        pred = predict(img_data[i])
        if pred==img_label[i]:
            cnt+=1
    return cnt/float(img_data.shape[0])

In [20]:
accuracy(img_data,img_label)

0.5643564356435643

# By Using Sckit classifier to predict

In [21]:
from sklearn import svm

In [22]:
svc = svm.SVC(C=1,kernel='linear')

In [23]:
svc.fit(img_data,img_label)
svc.score(img_data,img_label)

0.6658415841584159